In [ ]:
import numpy as np
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
import io
from sklearn.tree import DecisionTreeClassifier 
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from IPython.display import Image
from sklearn.tree import export_graphviz
from sklearn.model_selection import cross_val_score
from tqdm import tqdm_notebook as tqdm
from sklearn.externals.six import StringIO
import pydotplus
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt 
import seaborn as sns
import plotly.express as px 
# from cyberpandas import IPArray, to_ipaddress
import ipaddress
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from scipy.stats import nct
from sklearn.neighbors import LocalOutlierFactor

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)


# **DATA UPLOADING AND CLEANING**

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
from zipfile import ZipFile
filename="/content/drive/My Drive/MasterSoft/logbackups.zip"
with ZipFile(filename,"r") as zp:
  zp.extractall()
  print("Done") 

Done


In [ ]:
path = '/content/u_ex200701.log'
data = pd.read_csv(path , sep='\s+', skiprows=3, encoding='latin-1', error_bad_lines = False)

NameError: ignored

In [ ]:
lastHead=data.columns[-1]
data.drop([lastHead], axis=1,inplace=True)
newHead=data.columns[1:]
newHead=np.array(newHead)
newHead = np.append(newHead,lastHead)
data.columns=newHead

In [ ]:
data.dropna(inplace =True)
data = data[data['c-ip'] != "cs-username"] #extra header
data = data[data["cs-host"] != "localhost"] #localhost waste data
columns=['s-sitename','s-computername', 's-ip', 's-port', 'cs-version','cs-host','sc-substatus','sc-win32-status']
data = data.drop(columns,axis=1)
data = data.reset_index(drop=True)

In [ ]:
data

,date,time,cs-method,cs-uri-stem,cs-uri-query,cs-username,c-ip,cs(User-Agent),cs(Cookie),cs(Referer),sc-status,sc-bytes,cs-bytes,time-taken
0,2020-07-01,00:00:00,GET,/EKAP/Ortak/YeniIhaleAramaData.ashx,ES=&metot=isLogin,-,159.69.163.150,M-EKAP/1.9.4.1+(Android+5.1.1),ASP.NET_SessionId=has5g0gjfc5uzyl0uw4o1ky1;+BI...,-,200,387,317,7
1,2020-07-01,00:00:00,GET,/EKAP/Ortak/YeniIhaleAramaData.ashx,pageIndex=9&tabId=0ceef113-05a7-4476-f91c-6411...,-,159.69.163.150,M-EKAP/1.9.4.1+(Android+5.1.1),ASP.NET_SessionId=has5g0gjfc5uzyl0uw4o1ky1;+BI...,-,200,42331,470,595
2,2020-07-01,00:00:01,GET,/EKAP/Ortak/YeniIhaleAramaData.ashx,ES=&metot=isLogin,-,159.69.163.150,M-EKAP/1.9.4.1+(Android+5.1.1),ASP.NET_SessionId=has5g0gjfc5uzyl0uw4o1ky1;+BI...,-,200,387,317,4
3,2020-07-01,00:00:01,GET,/EKAP/Ortak/YeniIhaleAramaData.ashx,pageIndex=10&tabId=0ceef113-05a7-4476-f91c-641...,-,159.69.163.150,M-EKAP/1.9.4.1+(Android+5.1.1),ASP.NET_SessionId=has5g0gjfc5uzyl0uw4o1ky1;+BI...,-,200,424,471,526
4,2020-07-01,00:00:01,GET,/EKAP/Ortak/YeniIhaleAramaData.ashx,ES=&metot=isLogin,-,159.69.163.150,M-EKAP/1.9.4.1+(Android+5.1.1),ASP.NET_SessionId=has5g0gjfc5uzyl0uw4o1ky1;+BI...,-,200,387,317,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2681852,2020-07-01,23:59:58,GET,/EKAP/Ortak/BirBakistaIhale.aspx,IdareId=07aba124ac6f130c39fb59c55707f0a1b1ae52...,-,161.9.144.37,Mozilla/5.0+(Windows+NT+6.1;+Win64;+x64)+Apple...,BIGipServerGS-EKAP-WEB=rd2o0000000000000000000...,https://ekap.kik.gov.tr/EKAP/Ortak/BirBakistaI...,200,8206,1032,489
2681853,2020-07-01,23:59:58,GET,/EKAP/Ortak/BirBakistaIhale.aspx,IdareId=ae0e6b6b0f33c5c5213bca13fa1d636843451e...,-,88.255.228.26,Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+Appl...,linkYardimPortal2=1;+lbtnDuzenle201409=1;+css=...,https://ekap.kik.gov.tr/EKAP/Ortak/BirBakistaI...,200,9744,1220,70
2681854,2020-07-01,23:59:58,GET,/EKAP/Default.aspx,-,-,10.110.11.32,Mozilla/4.0+(compatible;+MSIE+6.0;+Windows+NT+...,-,-,200,101190,143,28
2681855,2020-07-01,23:59:59,GET,/EKAP/Ortak/BirBakistaIhale.aspx,IdareId=0ce336d8471edf0c33b4454d7bad214557b11d...,-,161.9.144.37,Mozilla/5.0+(Windows+NT+6.1;+Win64;+x64)+Apple...,BIGipServerGS-EKAP-WEB=rd2o0000000000000000000...,https://ekap.kik.gov.tr/EKAP/Ortak/BirBakistaI...,200,8121,1032,578


# **ONLY SELECTING THE ROW CONTAIN ".aspx"**

In [ ]:
dataASPX = data[data["cs-uri-stem"].str.contains(".aspx")]

In [ ]:
dataASPX

,date,time,cs-method,cs-uri-stem,cs-uri-query,cs-username,c-ip,cs(User-Agent),cs(Cookie),cs(Referer),sc-status,sc-bytes,cs-bytes,time-taken
5,2020-07-01,00:00:01,GET,/EKAP/Ortak/BirBakistaIhale.aspx,IdareId=97b19497907e190b6569d3a318d5d368f66c86...,17302892776,88.252.13.173,Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+Appl...,linkYardimPortal2=1;+_ga=GA1.3.1764455409.1583...,https://ekap.kik.gov.tr/EKAP/Ortak/BirBakistaI...,200,9788,1443,122
6,2020-07-01,00:00:02,GET,/EKAP/Ortak/BirBakistaIhale.aspx,IdareId=508d698f7f693d94de819da435e28da455d46a...,-,46.154.132.188,Mozilla/5.0+(Linux;+Android+8.0.0;+SAMSUNG+SM-...,linkYardimPortal2=1;+ASP.NET_SessionId=uq1z0yd...,https://ekap.kik.gov.tr/EKAP/Ortak/IhaleArama/...,200,9656,862,152
9,2020-07-01,00:00:04,POST,/EKAP/Istekli/KatilimYeterlik.aspx,ikn=2020%2f311296,36199271570,88.231.141.199,Mozilla/5.0+(Windows+NT+6.1;+Win64;+x64)+Apple...,linkYardimPortal2=1;+BIGipServerGS-EKAP-WEB=rd...,https://ekap.kik.gov.tr/EKAP/Istekli/KatilimYe...,200,23835,16096,194
12,2020-07-01,00:00:05,GET,/EKAP/Ortak/BirBakistaIhale.aspx,IdareId=bb79e8d2db6cae8a88c30c11c84c360554420c...,17302892776,88.252.13.173,Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+Appl...,linkYardimPortal2=1;+_ga=GA1.3.1764455409.1583...,https://ekap.kik.gov.tr/EKAP/Ortak/BirBakistaI...,200,9508,1443,107
15,2020-07-01,00:00:05,GET,/EKAP/Ortak/Feeds/EkapFeedRss.aspx,param=f29f0035e2ce136e0b7a747f7fdf6c209e5b5b1c...,-,185.68.222.5,Microsoft+Office/12.0+(Windows+NT+6.2;+Microso...,BIGipServerGS-EKAP-WEB=rd2o0000000000000000000...,-,200,11627,524,21088
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2681852,2020-07-01,23:59:58,GET,/EKAP/Ortak/BirBakistaIhale.aspx,IdareId=07aba124ac6f130c39fb59c55707f0a1b1ae52...,-,161.9.144.37,Mozilla/5.0+(Windows+NT+6.1;+Win64;+x64)+Apple...,BIGipServerGS-EKAP-WEB=rd2o0000000000000000000...,https://ekap.kik.gov.tr/EKAP/Ortak/BirBakistaI...,200,8206,1032,489
2681853,2020-07-01,23:59:58,GET,/EKAP/Ortak/BirBakistaIhale.aspx,IdareId=ae0e6b6b0f33c5c5213bca13fa1d636843451e...,-,88.255.228.26,Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+Appl...,linkYardimPortal2=1;+lbtnDuzenle201409=1;+css=...,https://ekap.kik.gov.tr/EKAP/Ortak/BirBakistaI...,200,9744,1220,70
2681854,2020-07-01,23:59:58,GET,/EKAP/Default.aspx,-,-,10.110.11.32,Mozilla/4.0+(compatible;+MSIE+6.0;+Windows+NT+...,-,-,200,101190,143,28
2681855,2020-07-01,23:59:59,GET,/EKAP/Ortak/BirBakistaIhale.aspx,IdareId=0ce336d8471edf0c33b4454d7bad214557b11d...,-,161.9.144.37,Mozilla/5.0+(Windows+NT+6.1;+Win64;+x64)+Apple...,BIGipServerGS-EKAP-WEB=rd2o0000000000000000000...,https://ekap.kik.gov.tr/EKAP/Ortak/BirBakistaI...,200,8121,1032,578


In [ ]:
dataASPX["cs-uri-stem"].value_counts()

/EKAP/Default.aspx                                                            85105
/EKAP/Ortak/BirBakistaIhale.aspx                                              81116
/EKAP/Teklif/IhaleDokumanDownloadEImzaliNet.aspx/CheckDokumanIndirmeStatus    50242
/EKAP/SystemLogin.aspx                                                        50002
/EKAP/Idare/Sihirbaz/DokumanHazirlama.aspx                                    47412
                                                                              ...  
/Oku.aspx                                                                         1
/YardimEkran/EBeyan/default.aspx                                                  1
/ekap/Oku.aspx                                                                    1
/EKAP/Yetki/IstekliKaydi.aspxadresi                                               1
/App_Master/Telerik.Web.UI.DialogHandler.aspx                                     1
Name: cs-uri-stem, Length: 458, dtype: int64

In [ ]:
dataASPX[dataASPX["cs-uri-stem"].str.contains("Default.aspx")]['cs-uri-stem'].value_counts()

/EKAP/Default.aspx                                       85105
/EKAP/Yardim/Default.aspx                                 1452
/EKAP/Yardim/Default.aspx/DosyaYolu                       1146
/EKAP/Yardim/Default.aspx/GetSSS                           437
/EKAP/Yardim/Default.aspx/GetYardimDokumanlari             437
/EKAP/Yardim/Default.aspx/GetDokuman                       436
/EKAP/Yardim/Default.aspx/TopluSSS                         430
/EKAP/Yardim/Default.aspx/GetKullaniciTip                  429
/EKAP/Yardim/Default.aspx/HeaderBul                        197
/EKAP/Istekli/EKAP/Yardim/Default.aspx                     156
/EKAP/Idare/EKAP/Yardim/Default.aspx                       103
/EKAP/Yardim/Default.aspx/Arama                            102
/EKAP/Yardim/Default.aspx/GetYasaKapsam                     78
/EKAP/Yardim/Default.aspx/GetMenu                           60
/EKAP/Yardim/Default.aspx/GetIHaleTur                       59
/EKAP/Teklif/EKAP/Yardim/Default.aspx                  

# **SELECTING "/EKAP/Ortak/BirBakistaIhale.aspx"**

In [ ]:
dataBirBakistaIhale = dataASPX[dataASPX["cs-uri-stem"]=="/EKAP/Ortak/BirBakistaIhale.aspx"]
dataBirBakistaIhale = dataBirBakistaIhale.reset_index(drop=True)

In [ ]:
dataBirBakistaIhale

,date,time,cs-method,cs-uri-stem,cs-uri-query,cs-username,c-ip,cs(User-Agent),cs(Cookie),cs(Referer),sc-status,sc-bytes,cs-bytes,time-taken
0,2020-07-01,00:00:01,GET,/EKAP/Ortak/BirBakistaIhale.aspx,IdareId=97b19497907e190b6569d3a318d5d368f66c86...,17302892776,88.252.13.173,Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+Appl...,linkYardimPortal2=1;+_ga=GA1.3.1764455409.1583...,https://ekap.kik.gov.tr/EKAP/Ortak/BirBakistaI...,200,9788,1443,122
1,2020-07-01,00:00:02,GET,/EKAP/Ortak/BirBakistaIhale.aspx,IdareId=508d698f7f693d94de819da435e28da455d46a...,-,46.154.132.188,Mozilla/5.0+(Linux;+Android+8.0.0;+SAMSUNG+SM-...,linkYardimPortal2=1;+ASP.NET_SessionId=uq1z0yd...,https://ekap.kik.gov.tr/EKAP/Ortak/IhaleArama/...,200,9656,862,152
2,2020-07-01,00:00:05,GET,/EKAP/Ortak/BirBakistaIhale.aspx,IdareId=bb79e8d2db6cae8a88c30c11c84c360554420c...,17302892776,88.252.13.173,Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+Appl...,linkYardimPortal2=1;+_ga=GA1.3.1764455409.1583...,https://ekap.kik.gov.tr/EKAP/Ortak/BirBakistaI...,200,9508,1443,107
3,2020-07-01,00:00:11,GET,/EKAP/Ortak/BirBakistaIhale.aspx,IdareId=9d38b0f795738d4397f5400440bbff259045fb...,-,46.154.132.188,Mozilla/5.0+(Linux;+Android+8.0.0;+SAMSUNG+SM-...,linkYardimPortal2=1;+ASP.NET_SessionId=uq1z0yd...,https://ekap.kik.gov.tr/EKAP/Ortak/BirBakistaI...,200,10115,1033,97
4,2020-07-01,00:00:13,GET,/EKAP/Ortak/BirBakistaIhale.aspx,IdareId=0409bf8284375665b1289d57f40e72906d1788...,-,46.154.132.188,Mozilla/5.0+(Linux;+Android+8.0.0;+SAMSUNG+SM-...,linkYardimPortal2=1;+ASP.NET_SessionId=uq1z0yd...,https://ekap.kik.gov.tr/EKAP/Ortak/IhaleArama/...,200,9544,862,156
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81111,2020-07-01,23:59:58,GET,/EKAP/Ortak/BirBakistaIhale.aspx,IdareId=1bd020d59d6c2b0ef14eeb7edb963cf9d851c5...,26458708572,176.42.110.141,Mozilla/5.0+(Windows+NT+6.1)+AppleWebKit/537.3...,linkYardimPortal2=1;+css=%2FEKAP%2FApp_Themes%...,https://ekap.kik.gov.tr/EKAP/Ortak/BirBakistaI...,200,9678,1426,131
81112,2020-07-01,23:59:58,GET,/EKAP/Ortak/BirBakistaIhale.aspx,IdareId=aecbdbf57790a8dfd5d0caebdc7e4a3e25213b...,-,46.155.245.35,Mozilla/5.0+(iPhone;+CPU+iPhone+OS+13_5_1+like...,browserDetection=browserDetection;+linkYardimP...,https://ekap.kik.gov.tr/EKAP/Ortak/IhaleArama/...,200,9615,767,176
81113,2020-07-01,23:59:58,GET,/EKAP/Ortak/BirBakistaIhale.aspx,IdareId=07aba124ac6f130c39fb59c55707f0a1b1ae52...,-,161.9.144.37,Mozilla/5.0+(Windows+NT+6.1;+Win64;+x64)+Apple...,BIGipServerGS-EKAP-WEB=rd2o0000000000000000000...,https://ekap.kik.gov.tr/EKAP/Ortak/BirBakistaI...,200,8206,1032,489
81114,2020-07-01,23:59:58,GET,/EKAP/Ortak/BirBakistaIhale.aspx,IdareId=ae0e6b6b0f33c5c5213bca13fa1d636843451e...,-,88.255.228.26,Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+Appl...,linkYardimPortal2=1;+lbtnDuzenle201409=1;+css=...,https://ekap.kik.gov.tr/EKAP/Ortak/BirBakistaI...,200,9744,1220,70


# **SELECTING SEVERAL FEATURES**

In [ ]:
columns = ["cs-method","sc-status","sc-bytes","cs-bytes","time-taken","cs-username"]
dataBirBakistaIhaleSelected = dataBirBakistaIhale[columns]
dataBirBakistaIhaleSelected

,cs-method,sc-status,sc-bytes,cs-bytes,time-taken,cs-username
0,GET,200,9788,1443,122,17302892776
1,GET,200,9656,862,152,-
2,GET,200,9508,1443,107,17302892776
3,GET,200,10115,1033,97,-
4,GET,200,9544,862,156,-
...,...,...,...,...,...,...
81111,GET,200,9678,1426,131,26458708572
81112,GET,200,9615,767,176,-
81113,GET,200,8206,1032,489,-
81114,GET,200,9744,1220,70,-


# **CONVERTING TO SUITABLE DATA TYPE**

In [ ]:
dataBirBakistaIhaleSelected["cs-username"].value_counts()

-              67402
26458708572     2446
16883908340      368
36925552096      327
31438587290      208
               ...  
11940013234        1
11080145932        1
64423255348        1
33217412946        1
39499707176        1
Name: cs-username, Length: 1104, dtype: int64

In [ ]:
dataBirBakistaIhaleSelected.loc[dataBirBakistaIhaleSelected['cs-username'] == "-", 'cs-username'] = "1" # User name anonymous

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
dataBirBakistaIhaleSelected["cs-username"].value_counts()

1              67402
26458708572     2446
16883908340      368
36925552096      327
31438587290      208
               ...  
32380400224        1
67849115494        1
20404487818        1
18793328348        1
39499707176        1
Name: cs-username, Length: 1104, dtype: int64

In [ ]:
integerColumns = ["sc-status","sc-bytes","cs-bytes","time-taken","cs-username"]
for column in integerColumns:
  dataBirBakistaIhaleSelected[column] = dataBirBakistaIhaleSelected[column].astype(int)
dataBirBakistaIhaleSelected['cs-method'] = (dataBirBakistaIhaleSelected['cs-method'] == 'GET').astype(int)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
dataBirBakistaIhaleSelected.loc[dataBirBakistaIhaleSelected['sc-status'] == 200, 'sc-statusBinary'] = 1 # status = 200
dataBirBakistaIhaleSelected.loc[dataBirBakistaIhaleSelected['sc-status'] != 200, 'sc-statusBinary'] = 0 #status = 302
dataBirBakistaIhaleSelected['sc-statusBinary'] = dataBirBakistaIhaleSelected['sc-statusBinary'].astype(int) #converting to integer from float

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:845: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas

In [ ]:
dataBirBakistaIhaleSelected.drop(["sc-status"],axis = 1,inplace = True)#dropping the sc-status row which contains 200 and 302

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
dataBirBakistaIhaleSelected.loc[dataBirBakistaIhaleSelected['cs-username'] !=1 , 'cs-username'] = 0 # Registered User

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
dataBirBakistaIhaleSelected

,cs-method,sc-bytes,cs-bytes,time-taken,cs-username,sc-statusBinary
0,1,9788,1443,122,0,1
1,1,9656,862,152,1,1
2,1,9508,1443,107,0,1
3,1,10115,1033,97,1,1
4,1,9544,862,156,1,1
...,...,...,...,...,...,...
81111,1,9678,1426,131,0,1
81112,1,9615,767,176,1,1
81113,1,8206,1032,489,1,1
81114,1,9744,1220,70,1,1


# **NORMALIZING SELECTED DATA**

In [ ]:
normalized_dataBirBakistaIhaleSelected = (dataBirBakistaIhaleSelected-dataBirBakistaIhaleSelected.min())/(dataBirBakistaIhaleSelected.max()-dataBirBakistaIhaleSelected.min())

In [ ]:
normalized_dataBirBakistaIhaleSelected

,cs-method,sc-bytes,cs-bytes,time-taken,cs-username,sc-statusBinary
0,1.0,0.039226,0.035469,0.000378,0.0,1.0
1,1.0,0.038697,0.018466,0.000487,1.0,1.0
2,1.0,0.038104,0.035469,0.000323,0.0,1.0
3,1.0,0.040536,0.023470,0.000287,1.0,1.0
4,1.0,0.038248,0.018466,0.000501,1.0,1.0
...,...,...,...,...,...,...
81111,1.0,0.038785,0.034971,0.000410,0.0,1.0
81112,1.0,0.038533,0.015686,0.000574,1.0,1.0
81113,1.0,0.032886,0.023441,0.001711,1.0,1.0
81114,1.0,0.039050,0.028943,0.000189,1.0,1.0


# **FINDING OPTIMUM PARAMETERS OF LOF**


In [ ]:
data=normalized_dataBirBakistaIhaleSelected.drop(["cs-method","cs-username"],axis = 1)

In [ ]:
data

,sc-bytes,cs-bytes,time-taken,sc-statusBinary
0,0.039226,0.035469,0.000378,1.0
1,0.038697,0.018466,0.000487,1.0
2,0.038104,0.035469,0.000323,1.0
3,0.040536,0.023470,0.000287,1.0
4,0.038248,0.018466,0.000501,1.0
...,...,...,...,...
81111,0.038785,0.034971,0.000410,1.0
81112,0.038533,0.015686,0.000574,1.0
81113,0.032886,0.023441,0.001711,1.0
81114,0.039050,0.028943,0.000189,1.0


In [ ]:
n_neighborsGrid= np.arange(5,80,5)

In [ ]:
n_neighborsGrid

array([ 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75])

In [ ]:
contaminationGrid = np.arange(0.1,0.5,0.05)

In [ ]:
contaminationGrid

array([0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45])

In [ ]:
n = len(data)
dfMeanInKC = pd.DataFrame(index = n_neighborsGrid,columns = contaminationGrid)
dfMeanOutKC = pd.DataFrame(index = n_neighborsGrid,columns = contaminationGrid)
dfVarianceInKC = pd.DataFrame(index = n_neighborsGrid,columns = contaminationGrid)
dfVarianceOutKC = pd.DataFrame(index = n_neighborsGrid,columns = contaminationGrid)
dfTtest = pd.DataFrame(index = n_neighborsGrid,columns = contaminationGrid)
cn_Values = np.zeros(len(contaminationGrid))

In [ ]:
for indexofC,c in enumerate(contaminationGrid):
  floor_cn=np.floor(c*n).astype(int)
  for indexofK,k in enumerate(n_neighborsGrid):
    clf = LocalOutlierFactor(n_neighbors=k, contamination = c)
    predict = clf.fit_predict(data)
    
    inliersIndex = np.where(predict == 1)
    outliersIndex = np.where(predict == -1)
    
    LOF = clf.negative_outlier_factor_*(-1)
    outliersLOF = LOF[outliersIndex]
    inliersLOF = LOF[inliersIndex]
    maxCN_inliersLOF = inliersLOF[inliersLOF.argsort()[(-1*floor_cn):][::-1]] #LOF scores of inliers for the top floor_cn 
    
    #natural log
    outliersLOF_naturalLog = np.log(outliersLOF)
    maxCN_inliersLOF_naturalLog = np.log(maxCN_inliersLOF)

    #variance of the logarithmic of outliers/inliers LOF scores
    varianceOutliersLOF = np.var(outliersLOF_naturalLog)
    varianceInliersLOF = np.var(maxCN_inliersLOF_naturalLog)

    #mean of the logarithmic of outliers/inliers LOF scores
    meanOutliersLOF = np.mean(outliersLOF_naturalLog)
    meanInliersLOF = np.mean(maxCN_inliersLOF_naturalLog)

    #filling dataFrames
    dfMeanInKC.at[k,c] = meanInliersLOF
    dfMeanOutKC.at[k,c] = meanOutliersLOF

    dfVarianceInKC.at[k,c] = varianceInliersLOF
    dfVarianceOutKC.at[k,c] = varianceOutliersLOF

  cn_Values[indexofC] = floor_cn

In [ ]:
dfTtest = (dfMeanOutKC-dfMeanInKC)/((1/floor_cn)*(dfVarianceOutKC+dfVarianceInKC))**(1/2)

#calculating of mean of inliers/outliers for every C value 
dfMeanInC = pd.DataFrame(dfMeanInKC.mean(),columns =['mean'])
dfMeanOutC = pd.DataFrame(dfMeanOutKC.mean(),columns =['mean'])

#calculating of variance of inliers/outliers for every C value 
dfVarianceInC = pd.DataFrame(dfVarianceInKC.mean(),columns =['mean'])
dfVarianceOutC = pd.DataFrame(dfVarianceOutKC.mean(),columns =['mean'])

#calculating of noncentrality parameter for C
dfNCPc = (dfMeanOutC-dfMeanInC)/(np.sqrt((1/floor_cn)*(dfVarianceOutC+dfVarianceInC)))

#calculating of degrees of freedom
dfDFc = (2*cn_Values)-2

#finding maximize values the standardized difference between outliers and inliers for every c value
dfKc = pd.DataFrame(dfTtest.max(),columns=["max"])


In [ ]:
ar_dfNCPc = np.asarray(dfNCPc)

In [ ]:
ar_dfDFc = np.asarray(dfDFc)

In [ ]:
ar_dfKc = np.asarray(dfKc)

In [ ]:
dfRandomVariables = pd.DataFrame()
for indexofC,c in enumerate(contaminationGrid):
   dfRandomVariables = dfRandomVariables.append(pd.Series(nct.rvs(ar_dfKc[indexofC], ar_dfNCPc[indexofC],size=8)), ignore_index=True)

In [ ]:
dfRandomVariables

,0,1,2,3,4,5,6,7
0,122.703375,117.763885,136.250938,128.796192,133.524866,134.003100,132.470433,129.545515
1,129.352042,133.305385,138.951901,120.005440,139.751518,119.976224,128.841072,143.380645
2,125.144216,141.481512,119.602311,125.914972,130.408958,125.566542,118.612346,133.390359
3,119.668705,127.807172,109.142093,116.388435,124.470973,110.365910,118.548632,124.987006
4,116.589577,117.264251,122.834416,128.845838,123.145729,116.280140,112.174458,120.515601
5,98.682695,122.250074,109.064529,125.555980,116.174732,109.582536,113.088835,112.919458
6,105.412864,100.186443,107.326710,113.287113,102.935580,107.320825,107.886747,100.518583
7,104.527953,107.888922,120.831371,125.104264,104.756060,102.829961,99.630100,110.815852


In [ ]:
dfKc

,max
0.10,175.027008
0.15,169.288105
0.20,161.293618
0.25,153.368070
0.30,147.815937
0.35,143.149449
0.40,139.309500
0.45,136.425462


In [ ]:
dfTtest

,0.10,0.15,0.20,0.25,0.30,0.35,0.40,0.45
5,84.2876,76.8593,72.6104,69.5862,67.24,65.4119,63.7485,62.6269
10,145.083,142.162,138.747,135.519,132.486,129.744,127.396,125.721
15,170.7,164.86,158.902,153.082,147.816,143.149,139.309,136.425
20,161.546,158.002,153.071,147.577,142.31,137.658,133.856,130.943
25,159.967,156.323,151.257,145.893,140.911,136.397,132.556,129.531
30,167.518,161.743,156.045,150.365,144.929,139.978,135.773,132.391
35,162.834,158.329,153.381,147.905,142.575,137.785,133.608,130.229
40,162.021,158.331,153.565,147.78,142.107,137.082,132.795,129.327
45,164.295,159.558,154.182,147.848,141.782,136.607,132.42,129.056
50,165.62,160.612,155.07,148.451,142.136,136.932,132.745,129.421


# **LOF TEST**

In [ ]:
clf = LocalOutlierFactor(n_neighbors=70, contamination = 0.1)
predict = clf.fit_predict(data)

In [ ]:
dataLabelled = dataBirBakistaIhaleSelected.drop(["cs-method","cs-username"],axis=1)

In [ ]:
dataLabelled['label'] = predict 

In [ ]:
dataLabelled

,sc-bytes,cs-bytes,time-taken,sc-statusBinary,label
0,9788,1443,122,1,-1
1,9656,862,152,1,1
2,9508,1443,107,1,1
3,10115,1033,97,1,-1
4,9544,862,156,1,1
...,...,...,...,...,...
81111,9678,1426,131,1,1
81112,9615,767,176,1,1
81113,8206,1032,489,1,1
81114,9744,1220,70,1,1


In [ ]:
df_status200 = dataLabelled[dataLabelled['sc-statusBinary']==1]

In [ ]:
df_statusOther = dataLabelled[dataLabelled['sc-statusBinary']==0]

In [ ]:
df_statusOther['label'].value_counts()

-1    58
Name: label, dtype: int64

In [ ]:
df_status200['label'].value_counts()

 1    73004
-1     8054
Name: label, dtype: int64

In [ ]:
df_status200[df_status200['label']==-1].head(10)

,sc-bytes,cs-bytes,time-taken,sc-statusBinary,label
0,9788,1443,122,1,-1
3,10115,1033,97,1,-1
7,8301,1122,61,1,-1
10,9876,844,88,1,-1
23,8764,1162,62,1,-1
24,9585,1113,107,1,-1
39,8224,922,561,1,-1
51,7926,1079,74,1,-1
55,8220,1079,91,1,-1
56,8331,1079,75,1,-1


In [ ]:
df_status200[df_status200['label']==1].head(10)

,sc-bytes,cs-bytes,time-taken,sc-statusBinary,label
1,9656,862,152,1,1
2,9508,1443,107,1,1
4,9544,862,156,1,1
5,8159,1113,62,1,1
6,9870,862,186,1,1
8,8280,1377,94,1,1
9,9789,1033,106,1,1
11,8099,1113,79,1,1
12,9808,862,148,1,1
13,9763,1033,96,1,1


In [ ]:
LOF = pd.DataFrame(np.asarray(clf.negative_outlier_factor_*(-1)))

In [ ]:
LOF.iloc[39]

0    3.482163
Name: 39, dtype: float64

In [ ]:
LOF.iloc[8]

0    1.207085
Name: 8, dtype: float64

In [ ]:
unexpected = LOF.index[(LOF[0]>1.2) & (LOF[0]<1.3)].tolist()

In [ ]:
dataLabelled.iloc[unexpected]

,sc-bytes,cs-bytes,time-taken,sc-statusBinary,label
8,8280,1377,94,1,1
32,8679,1162,69,1,1
40,10334,1093,54,1,1
72,2411,3204,46,1,-1
73,9953,1460,125,1,1
...,...,...,...,...,...
81011,8799,861,122,1,1
81014,8325,1032,550,1,1
81056,8775,861,125,1,1
81062,8425,1032,603,1,1


In [ ]:
LOF.iloc[40]

0    1.266664
Name: 40, dtype: float64

In [ ]:
df_status200[(df_status200['label']==1) & (df_status200['sc-bytes']>8200)]

,sc-bytes,cs-bytes,time-taken,sc-statusBinary,label
1,9656,862,152,1,1
2,9508,1443,107,1,1
4,9544,862,156,1,1
6,9870,862,186,1,1
8,8280,1377,94,1,1
...,...,...,...,...,...
81109,10319,1426,147,1,1
81111,9678,1426,131,1,1
81112,9615,767,176,1,1
81113,8206,1032,489,1,1


In [ ]:
# def findingOptimumParameterLOF(n_neighborsGrid,contaminationGrid,data):

n = len(data)
dfMeanInKC = pd.Dataframe(index = n_neighborsGrid,columns = contaminationGrid)
dfMeanOutKC = pd.Dataframe(index = n_neighborsGrid,columns = contaminationGrid)
dfVarianceInKC = pd.Dataframe(index = n_neighborsGrid,columns = contaminationGrid)
dfVarianceOutKC = pd.Dataframe(index = n_neighborsGrid,columns = contaminationGrid)
dfTtest = pd.Dataframe(index = n_neighborsGrid,columns = contaminationGrid)
cn_Values = np.zeros(len(contaminationGrid))

for indexofC,c in enumerate(contaminationGrid):
  floor_cn=np.floor(c*n).astype(int)
  for indexofK,k in enumerate(n_neighborsGrid):
    clf = LocalOutlierFactor(n_neighbors=k, contamination = c)
    predict = clf.fit_predict(data)
    
    inlinersIndex = np.where(predict == 1)
    outliersIndex = np.where(predict == -1)
    
    LOF = clf.negative_outlier_factor_*(-1)
    outliersLOF = LOF[outliersIndex]
    inliersLOF = LOF[inliersIndex]
    maxCN_inliersLOF = inliersLOF[inliersLOF.argsort()[(-1*floor_cn):][::-1]] #LOF scores of inliers for the top floor_cn 
    
    #natural log
    outliersLOF_naturalLog = np.log(outliersLOF)
    maxCN_inliersLOF_naturalLog = np.log(maxCN_inliersLOF)

    #variance of the logarithmic of outliers/inliers LOF scores
    varianceOutliersLOF = np.var(outliersLOF_naturalLog)
    varianceInliersLOF = np.var(maxCN_inliersLOF_naturalLog)

    #mean of the logarithmic of outliers/inliers LOF scores
    meanOutliersLOF = np.mean(outliersLOF_naturalLog)
    meanInliersLOF = np.mean(maxCN_inliersLOF_naturalLog)

    #filling dataFrames
    dfMeanInKC.at[c,k] = meanInliersLOF
    dfMeanOutKC.at[c,k] = meanOutliersLOF

    dfVarianceInKC.at[c,k] = varianceInliersLOF
    dfVarianceOutKC.at[c,k] = varianceOutliersLOF

  cn_Values[indexofC] = floor_cn

# calculating of the standardized difference in the mean log local outlier factor scores between outliers and inliers
dfTtest = (dfMeanOutKC-dfMeanInKC)/(np.sqrt((1/floor_cn)*(dfVarianceOutKC+dfVarianceInKC)))

#calculating of mean of inliers/outliers for every C value 
dfMeanInC = pd.DataFrame(dfMeanInKC.mean(),columns =['mean'])
dfMeanOutC = pd.DataFrame(dfMeanOutKC.mean(),columns =['mean'])

#calculating of variance of inliers/outliers for every C value 
dfVarianceInC = pd.DataFrame(dfVarianceInKC.mean(),columns =['mean'])
dfVarianceOutC = pd.DataFrame(dfVarianceOutKC.mean(),columns =['mean'])

#calculating of noncentrality parameter for C
dfNCPc = (dfMeanOutC-dfMeanInC)/(np.sqrt((1/floor_cn)*(dfVarianceOutC+dfVarianceInC)))

#calculating of degrees of freedom
dfDFc = (2*cn_Values)-2

#finding maximize values the standardized difference between outliers and inliers for every c value
dfKc = pd.DataFrame(dfTtest.max(),columns=["max"])

#
for indexofC,c in enumerate(contaminationGrid):
  special.nctdtr(dfDFc[0][indexofC], dfNCPc["mean"][c], dfKc["max"][c])